In [1]:
# Cargamos las librerias que vamos a utilizar
import pandas as pd
import os
import ast
from ast import literal_eval
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

1.  def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [2]:
aus_User_Items = os.path.join('Datasets-Limpios', 'user_items.parquet')
output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')

In [3]:
output_steam_games_parquet = pd.read_parquet(output_Steam_Games)

In [4]:
output_steam_games_parquet.head()

,genres,tags,release_date,price,item_id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
4,"[Action, Indie, Casual, Sports]",Log Challenge,NaT,2.99,773570,Desconocido


In [5]:
len(output_steam_games_parquet)

31994

In [6]:
user_items = pd.read_parquet(aus_User_Items)

In [7]:
user_items.head()

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,277
2,30,Day of Defeat,7,76561197970982479,277
3,40,Deathmatch Classic,0,76561197970982479,277
4,50,Half-Life: Opposing Force,0,76561197970982479,277


In [8]:
len(user_items)

5094092

In [9]:
columnas_file1 = user_items[['playtime_forever','item_id']]
output_steam_games_parquet = output_steam_games_parquet.rename(columns={'id': 'item_id'})
columnas_file2 = output_steam_games_parquet[['genres', 'release_date', 'item_id']]

In [10]:
merged_data = pd.merge(columnas_file1, columnas_file2, left_on='item_id', right_on='item_id', how='inner')

In [11]:
merged_data.head()

,playtime_forever,item_id,genres,release_date
0,6,10,[Action],2000-11-01
1,0,10,[Action],2000-11-01
2,0,10,[Action],2000-11-01
3,93,10,[Action],2000-11-01
4,108,10,[Action],2000-11-01


In [12]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4244767 entries, 0 to 4244766
Data columns (total 4 columns):
 #   Column            Dtype         
---  ------            -----         
 0   playtime_forever  int64         
 1   item_id           int64         
 2   genres            object        
 3   release_date      datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 129.5+ MB


In [13]:
len(merged_data)

4244767

In [14]:
merged_data.isnull().sum()

playtime_forever         0
item_id                  0
genres                   0
release_date        101415
dtype: int64

In [15]:
merged_data = merged_data.dropna(subset=['release_date'])

In [16]:
merged_data.isnull().sum()

playtime_forever    0
item_id             0
genres              0
release_date        0
dtype: int64

In [17]:
len(merged_data)

4143352

In [18]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4143352 entries, 0 to 4244765
Data columns (total 4 columns):
 #   Column            Dtype         
---  ------            -----         
 0   playtime_forever  int64         
 1   item_id           int64         
 2   genres            object        
 3   release_date      datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 158.1+ MB


In [19]:
merged_data['release_date'] = pd.to_datetime(merged_data['release_date'], errors='coerce', format='%Y', exact=False)

In [20]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4143352 entries, 0 to 4244765
Data columns (total 4 columns):
 #   Column            Dtype         
---  ------            -----         
 0   playtime_forever  int64         
 1   item_id           int64         
 2   genres            object        
 3   release_date      datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 158.1+ MB


In [21]:
merged_data = merged_data.explode('genres')

In [22]:
merged_data['genres'].unique()

array(['Action', 'Strategy', 'RPG', 'Indie', 'Casual', 'Simulation',
       'Adventure', 'Racing', 'Sports', 'Early Access', 'Free to Play',
       'Massively Multiplayer', nan, 'Utilities',
       'Animation &amp; Modeling', 'Design &amp; Illustration',
       'Education', 'Web Publishing', 'Video Production',
       'Audio Production', 'Software Training', 'Photo Editing'],
      dtype=object)

In [23]:
merged_data.head()

,playtime_forever,item_id,genres,release_date
0,6,10,Action,2000-11-01
1,0,10,Action,2000-11-01
2,0,10,Action,2000-11-01
3,93,10,Action,2000-11-01
4,108,10,Action,2000-11-01


In [24]:
grouped_data = merged_data.groupby(['genres', merged_data['release_date'].dt.year])['playtime_forever'].sum().reset_index()

In [25]:
grouped_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            343 non-null    object
 1   release_date      343 non-null    int32 
 2   playtime_forever  343 non-null    int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 6.8+ KB


In [26]:
grouped_data.tail()

,genres,release_date,playtime_forever
338,Web Publishing,2013,333678
339,Web Publishing,2014,33641
340,Web Publishing,2015,348673
341,Web Publishing,2016,136
342,Web Publishing,2017,9382


In [27]:
grouped_data.genres.isnull().sum()

0

In [28]:
grouped_data[grouped_data.genres == ''].count()

genres              0
release_date        0
playtime_forever    0
dtype: int64

In [29]:
grouped_data.head()

,genres,release_date,playtime_forever
0,Action,1983,3473
1,Action,1984,384
2,Action,1988,16001
3,Action,1989,607
4,Action,1990,18335


In [30]:
grouped_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            343 non-null    object
 1   release_date      343 non-null    int32 
 2   playtime_forever  343 non-null    int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 6.8+ KB


In [31]:
grouped_data['genres'].unique()

array(['Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing',
       'Simulation', 'Software Training', 'Sports', 'Strategy',
       'Utilities', 'Video Production', 'Web Publishing'], dtype=object)

In [32]:
grouped_data['genres'] = grouped_data['genres'].replace('Design &amp; Illustration', 'Design & Illustration')
grouped_data['genres'] = grouped_data['genres'].replace('Animation &amp; Modeling', 'Animation & Modeling')

In [33]:
grouped_data.to_csv('consulta1.csv', index=False)

2.    def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [34]:
aus_User_Items = os.path.join('Datasets-Limpios', 'user_items.parquet')
output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')

In [35]:
output_games = pd.read_parquet(output_Steam_Games)
user_items = pd.read_parquet(aus_User_Items)

In [36]:
output_games.head()

,genres,tags,release_date,price,item_id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
4,"[Action, Indie, Casual, Sports]",Log Challenge,NaT,2.99,773570,Desconocido


In [37]:
user_items.head()

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6,76561197970982479,277
1,20,Team Fortress Classic,0,76561197970982479,277
2,30,Day of Defeat,7,76561197970982479,277
3,40,Deathmatch Classic,0,76561197970982479,277
4,50,Half-Life: Opposing Force,0,76561197970982479,277


In [38]:
dropoutput = output_games.drop(['tags', 'developer', 'price'],axis=1)
dropuseritems = user_items.drop(['item_name','items_count'],axis=1)

In [39]:
merged_data=pd.merge(dropuseritems,dropoutput,left_on='item_id', right_on='item_id', how='right')
merged_data.drop(['item_id'], axis=1, inplace=True)

In [40]:
merged_data.head()

,playtime_forever,user_id,genres,release_date
0,NaN,NaN,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04
1,NaN,NaN,"[Free to Play, Indie, RPG, Strategy]",2018-01-04
2,NaN,NaN,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24
3,NaN,NaN,"[Action, Adventure, Casual]",2017-12-07
4,NaN,NaN,"[Action, Indie, Casual, Sports]",NaT


In [41]:
len(merged_data)

4267571

In [42]:
merged_data.isnull().sum()

playtime_forever     22804
user_id              22804
genres                   0
release_date        103291
dtype: int64

In [43]:
merged_data.dropna(inplace=True)

In [44]:
merged_data.isnull().sum()

playtime_forever    0
user_id             0
genres              0
release_date        0
dtype: int64

In [45]:
len(merged_data)

4143352

In [46]:
merged_data.head()

,playtime_forever,user_id,genres,release_date
27,5.0,UTNerd24,"[Action, Indie, Racing]",1997-06-30
28,0.0,I_DID_911_JUST_SAYING,"[Action, Indie, Racing]",1997-06-30
29,0.0,76561197962104795,"[Action, Indie, Racing]",1997-06-30
30,0.0,r3ap3r78,"[Action, Indie, Racing]",1997-06-30
31,13.0,saint556,"[Action, Indie, Racing]",1997-06-30


In [47]:
merged_data['playtime_forever'] = merged_data['playtime_forever'].astype(int)

In [48]:
merged_data.head()

,playtime_forever,user_id,genres,release_date
27,5,UTNerd24,"[Action, Indie, Racing]",1997-06-30
28,0,I_DID_911_JUST_SAYING,"[Action, Indie, Racing]",1997-06-30
29,0,76561197962104795,"[Action, Indie, Racing]",1997-06-30
30,0,r3ap3r78,"[Action, Indie, Racing]",1997-06-30
31,13,saint556,"[Action, Indie, Racing]",1997-06-30


In [49]:
merged_data = merged_data.explode('genres')

In [50]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9840519 entries, 27 to 4267553
Data columns (total 4 columns):
 #   Column            Dtype         
---  ------            -----         
 0   playtime_forever  int32         
 1   user_id           object        
 2   genres            object        
 3   release_date      datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 337.8+ MB


In [51]:
merged_data.head()

,playtime_forever,user_id,genres,release_date
27,5,UTNerd24,Action,1997-06-30
27,5,UTNerd24,Indie,1997-06-30
27,5,UTNerd24,Racing,1997-06-30
28,0,I_DID_911_JUST_SAYING,Action,1997-06-30
28,0,I_DID_911_JUST_SAYING,Indie,1997-06-30


In [52]:
max_hours_df = merged_data.loc[merged_data.groupby(['genres', 'release_date'])['playtime_forever'].idxmax()]

In [53]:
max_hours_df = max_hours_df[['genres', 'release_date', 'user_id', 'playtime_forever']]

In [54]:
consulta2 = max_hours_df.groupby(['genres', 'release_date', 'user_id'])['playtime_forever'].sum().reset_index()

In [55]:
consulta2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14270 entries, 0 to 14269
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   genres            14270 non-null  object        
 1   release_date      14270 non-null  datetime64[ns]
 2   user_id           14270 non-null  object        
 3   playtime_forever  14270 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 390.3+ KB


In [56]:
consulta2.head(20).sort_values('playtime_forever',ascending=False)

,genres,release_date,user_id,playtime_forever
15,Action,1993-01-01,spooke512,3624
7,Action,1991-06-16,76561198056997935,2283
5,Action,1990-12-14,76561198041356854,1424
19,Action,1993-10-10,76561198036455045,1360
2,Action,1988-12-23,DownSyndromeKid,1328
12,Action,1992-03-01,76561198041127602,1210
0,Action,1983-06-19,76561197966936422,993
13,Action,1992-05-01,akaNoodleBox,874
3,Action,1989-01-01,starfleetbrat,494
1,Action,1984-04-29,tankmannz,354


In [57]:
consulta2.to_csv('consulta2.csv', index=False)

3.    def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [58]:
aus_User_Reviews = os.path.join('Datasets-Limpios', 'user_reviews.parquet')
output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')

In [59]:
user_reviews = pd.read_parquet(aus_User_Reviews)
output_games = pd.read_parquet(output_Steam_Games)

In [60]:
user_reviews.columns

Index(['item_id', 'recommend', 'review', 'user_id', 'year_posted'], dtype='object')

In [61]:
user_reviews.head()

,item_id,recommend,review,user_id,year_posted
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011
3,251610,True,I know what you think when you see this title ...,js41637,2014
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013


In [62]:
nltk.download('vader_lexicon')

def analyze_sentiment(review):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(review)['compound']
    
    if sentiment_score >= 0.05:
        return 2  # Positivo
    elif sentiment_score <= -0.05:
        return 0  # Malo
    else:
        return 1  # Neutral
    
user_reviews['sentiment_analysis'] = user_reviews['review'].apply(lambda x: analyze_sentiment(x) if pd.notnull(x) else 1)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\BunkiSpeeder\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [63]:
user_reviews[user_reviews['sentiment_analysis'] == 2]

,item_id,recommend,review,user_id,year_posted,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011,2
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011,2
3,251610,True,I know what you think when you see this title ...,js41637,2014,2
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013,2
...,...,...,...,...,...,...
58426,70,True,a must have classic from steam definitely wort...,76561198312638244,2016,2
58427,362890,True,this game is a perfect remake of the original ...,76561198312638244,2016,2
58428,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,2016,2
58429,730,True,:D,LydiaMorley,2016,2


In [64]:
user_reviews[user_reviews['sentiment_analysis'] == 1]

,item_id,recommend,review,user_id,year_posted,sentiment_analysis
9,263360,True,"Random drops and random quests, with stat poin...",evcentric,2014,1
18,211420,True,Git gud,maplemage,2014,1
22,249130,True,This game is Marvellous.,Wackky,2014,1
23,207610,True,"It reminds me of that TV Show called ""The Walk...",Wackky,2012,1
27,730,True,ZIKA DO BAILE,76561198079601835,2016,1
...,...,...,...,...,...,...
58407,220090,True,GET THIS GAME AND CHAPTER TWO!!!!!!!!!!!!!!!!!...,MeloncraftLP,2016,1
58408,262850,True,THIS GAME!!!!!!!!!!!!!!!!!! WOOOOOOOOOOOOOOOOO...,MeloncraftLP,2016,1
58411,298110,False,"uplay, everytime",943525,2016,1
58418,427730,True,dont ask,sexyawp,2016,1


In [65]:
user_reviews[user_reviews['sentiment_analysis'] == 0]

,item_id,recommend,review,user_id,year_posted,sentiment_analysis
16,207610,True,The ending to this game is.... ♥♥♥♥♥♥♥.... Jus...,doctr,2012,0
29,72850,True,"Killed the Emperor, nobody cared and got away ...",76561198089393905,2015,0
43,1250,True,"Compared to Left 4 Dead 2, this game REALLY gi...",DJKamBer,2013,0
55,238090,True,"One of the best sequals i have every seen, reb...",starkillershadow553,2015,0
56,242130,True,"If you're looking for a no frills, Arcade/Simu...",starkillershadow553,2016,0
...,...,...,...,...,...,...
58394,391460,False,Bad,iwishihadaids,2016,0
58404,381210,True,"Fix your ♥♥♥♥ing game, de rank me for no reaso...",76561198270958927,2016,0
58413,413150,True,"It's pretty fun, very very similiar to harvest...",vinquility,2016,0
58415,304930,True,I really like Unturned but the thing that i wa...,76561198289386531,2016,0


In [66]:
output_games.head()

,genres,tags,release_date,price,item_id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
4,"[Action, Indie, Casual, Sports]",Log Challenge,NaT,2.99,773570,Desconocido


In [67]:
output_games2 = output_games[['item_id', 'tags']]

In [68]:
output_games2.head()

,item_id,tags
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,773570,Log Challenge


In [69]:
user_reviews.columns

Index(['item_id', 'recommend', 'review', 'user_id', 'year_posted',
       'sentiment_analysis'],
      dtype='object')

In [70]:
len(user_reviews)

58431

In [71]:
columnas_file1 = output_games2[['item_id','tags']]
columnas_file2 = user_reviews[['item_id', 'recommend','review','user_id','year_posted','sentiment_analysis']]

In [72]:
merged_reviews = pd.merge(columnas_file1, columnas_file2, left_on='item_id', right_on='item_id', how='inner')

In [73]:
merged_reviews.head()

,item_id,tags,recommend,review,user_id,year_posted,sentiment_analysis
0,282010,Carmageddon Max Pack,True,"Oh, where do i start? DOS...",InstigatorAU,2016,1
1,70,Half-Life,True,ADVERTENCIA:Este Análisis contiene posibles Sp...,EizanAratoFujimaki,2015,2
2,70,Half-Life,True,If you own any half life but not this one that...,GamerFag,2011,0
3,70,Half-Life,True,this game has had me hooked for over 4 years a...,76561198020928326,2014,2
4,70,Half-Life,True,"This is, by far my most favourite game I have ...",Bluegills,2013,2


In [74]:
merged_reviews = merged_reviews.drop(['item_id', 'user_id','review'], axis=1)

In [75]:
merged_reviews.head()

,tags,recommend,year_posted,sentiment_analysis
0,Carmageddon Max Pack,True,2016,1
1,Half-Life,True,2015,2
2,Half-Life,True,2011,0
3,Half-Life,True,2014,2
4,Half-Life,True,2013,2


In [76]:
merged_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53196 entries, 0 to 53195
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                53196 non-null  object
 1   recommend           53196 non-null  bool  
 2   year_posted         53196 non-null  int64 
 3   sentiment_analysis  53196 non-null  int64 
dtypes: bool(1), int64(2), object(1)
memory usage: 1.3+ MB


In [77]:
df_filtered = merged_reviews[merged_reviews['recommend'] == True]

In [78]:
df_grouped = df_filtered.groupby('tags').agg({'sentiment_analysis': 'count', 'year_posted': 'first'}).reset_index()

In [79]:
df_grouped.rename(columns={'sentiment_analysis': 'total_sentiment_analysis'}, inplace=True)

In [80]:
df_grouped

,tags,total_sentiment_analysis,year_posted
0,//N.P.P.D. RUSH//- The milk of Ultraviolet,1,2015
1,0RBITALIS,1,2014
2,"10,000,000",3,2015
3,100% Orange Juice,12,2015
4,100% Orange Juice - Krila & Kae Character Pack,1,2016
...,...,...,...
2864,the static speaks my name,13,2015
2865,theBlu,1,2016
2866,theHunter Classic,32,2015
2867,theHunter: Primal,13,2015


In [81]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2869 entries, 0 to 2868
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   tags                      2869 non-null   object
 1   total_sentiment_analysis  2869 non-null   int64 
 2   year_posted               2869 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 67.4+ KB


In [82]:
df_grouped.to_csv('consulta3.csv', index=False)

In [83]:
user_reviews.head()

,item_id,recommend,review,user_id,year_posted,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011,2
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011,2
3,251610,True,I know what you think when you see this title ...,js41637,2014,2
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013,2


In [84]:
user_reviews2 = user_reviews.drop(['review', 'user_id'], axis=1)

In [85]:
user_reviews2.head()

,item_id,recommend,year_posted,sentiment_analysis
0,1250,True,2011,2
1,22200,True,2011,2
2,43110,True,2011,2
3,251610,True,2014,2
4,227300,True,2013,2


In [86]:
fil = (user_reviews2['recommend'] == False) & (user_reviews2['sentiment_analysis'] == 0)
user_reviews2 = user_reviews2[fil]

In [87]:
user_reviews2.head()

,item_id,recommend,year_posted,sentiment_analysis
62,359320,False,2015,0
132,344760,False,2015,0
183,437220,False,2016,0
226,299740,False,2016,0
241,48240,False,2014,0


In [88]:
user_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2880 entries, 62 to 58394
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   item_id             2880 non-null   int64
 1   recommend           2880 non-null   bool 
 2   year_posted         2880 non-null   int64
 3   sentiment_analysis  2880 non-null   int64
dtypes: bool(1), int64(3)
memory usage: 92.8 KB


In [89]:
user_reviews2 = user_reviews2.drop(columns='recommend')

In [90]:
user_reviews2

,item_id,year_posted,sentiment_analysis
62,359320,2015,0
132,344760,2015,0
183,437220,2016,0
226,299740,2016,0
241,48240,2014,0
...,...,...,...
58265,295110,2016,0
58292,305920,2015,0
58341,570,2016,0
58351,370240,2015,0


In [91]:
grouped_reviews2 = user_reviews2.groupby(['year_posted', 'item_id']).agg({'sentiment_analysis': 'count'}).reset_index()

In [92]:
grouped_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1349 entries, 0 to 1348
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   year_posted         1349 non-null   int64
 1   item_id             1349 non-null   int64
 2   sentiment_analysis  1349 non-null   int64
dtypes: int64(3)
memory usage: 31.7 KB


In [93]:
grouped_reviews2.head()

,year_posted,item_id,sentiment_analysis
0,2011,440,1
1,2011,18700,2
2,2011,63940,1
3,2011,91310,1
4,2011,105400,1


In [94]:
output_games_columnasf4 = output_games[['item_id', 'developer']]

In [95]:
output_games_columnasf4.head()

,item_id,developer
0,761140,Kotoshiro
1,643980,Secret Level SRL
2,670290,Poolians.com
3,767400,彼岸领域
4,773570,Desconocido


In [96]:
merged_df = pd.merge(grouped_reviews2, output_games_columnasf4[['item_id', 'developer']], on='item_id', how='left')

In [97]:
merged_df.dropna()

,year_posted,item_id,sentiment_analysis,developer
0,2011,440,1,Valve
1,2011,18700,2,Broken Rules
2,2011,63940,1,1C Company
5,2012,440,1,Valve
7,2012,42920,1,NeoCoreGames
...,...,...,...,...
1342,2016,480630,1,Bethesda Game Studios
1343,2016,488280,1,AZAMATIKA
1345,2016,493370,1,Playsaurus
1346,2016,501760,1,Kiddy


In [98]:
merged_df.head()

,year_posted,item_id,sentiment_analysis,developer
0,2011,440,1,Valve
1,2011,18700,2,Broken Rules
2,2011,63940,1,1C Company
3,2011,91310,1,NaN
4,2011,105400,1,NaN


In [99]:
unknown_count = merged_df['developer'].value_counts().get('Desconocido', 0)
print(f"Cantidad de 'Desconocido': {unknown_count}")

Cantidad de 'Desconocido': 35


In [100]:
merged_df = merged_df.loc[merged_df['developer'] != 'Desconocido']

In [101]:
merged_df.dropna()

,year_posted,item_id,sentiment_analysis,developer
0,2011,440,1,Valve
1,2011,18700,2,Broken Rules
2,2011,63940,1,1C Company
5,2012,440,1,Valve
7,2012,42920,1,NeoCoreGames
...,...,...,...,...
1342,2016,480630,1,Bethesda Game Studios
1343,2016,488280,1,AZAMATIKA
1345,2016,493370,1,Playsaurus
1346,2016,501760,1,Kiddy


In [102]:
merged_df.to_csv('consulta4.csv', index=False)

In [103]:
user_reviews.head()

,item_id,recommend,review,user_id,year_posted,sentiment_analysis
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,2011,2
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011,2
3,251610,True,I know what you think when you see this title ...,js41637,2014,2
4,227300,True,For a simple (it's actually not all that simpl...,js41637,2013,2


In [104]:
output_games.head()

,genres,tags,release_date,price,item_id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,0.00,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,0.00,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
4,"[Action, Indie, Casual, Sports]",Log Challenge,NaT,2.99,773570,Desconocido


In [105]:
output_games_f5 = output_games.drop(columns=['genres', 'tags', 'release_date', 'price'])

In [106]:
output_games_f5.head()

,item_id,developer
0,761140,Kotoshiro
1,643980,Secret Level SRL
2,670290,Poolians.com
3,767400,彼岸领域
4,773570,Desconocido


In [107]:
output_games_f5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    31994 non-null  int32 
 1   developer  31994 non-null  object
dtypes: int32(1), object(1)
memory usage: 375.1+ KB


In [108]:
user_reviews_f5 = user_reviews.drop(columns=['recommend', 'review', 'user_id', 'year_posted'])

In [109]:
user_reviews_f5.head()

,item_id,sentiment_analysis
0,1250,2
1,22200,2
2,43110,2
3,251610,2
4,227300,2


In [110]:
user_reviews_f5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   item_id             58431 non-null  int64
 1   sentiment_analysis  58431 non-null  int64
dtypes: int64(2)
memory usage: 913.1 KB


In [111]:
merged_df = pd.merge(user_reviews_f5, output_games_f5, on='item_id', how='inner')

In [112]:
merged_df.head()

,item_id,sentiment_analysis,developer
0,1250,2,Tripwire Interactive
1,1250,2,Tripwire Interactive
2,1250,0,Tripwire Interactive
3,1250,1,Tripwire Interactive
4,1250,1,Tripwire Interactive


In [113]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53196 entries, 0 to 53195
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             53196 non-null  int64 
 1   sentiment_analysis  53196 non-null  int64 
 2   developer           53196 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.2+ MB


In [114]:
merged_df.to_csv('consulta5.csv', index=False)